# <b>Bayesian Logistic Regression - Metropolis-Hastings</b>

# Aims
<ul>
<li>To implement the MH algorithm.</li>
<li>To use it to compute classification probabilities.</li>
</ul>
<br><br>
Let's start importing some useful modules.

In [38]:
import math

import numpy                 as np
import matplotlib.pyplot     as plt
import seaborn               as sns
import plotly.plotly         as py
import plotly.graph_objs     as go
import scipy.io              as sio
import pandas                as pd
import plotly.figure_factory as ff

from scipy         import optimize, linalg
from pprint        import pprint
from jupyterthemes import jtplot


jtplot.style('grade3', context='notebook', fscale=1.4)
jtplot.style(ticks=True, grid=False)
jtplot.figsize(x=15, y=10)

# Metropolis-Hastings
In this lab, we’re going to implement the Metropolis-Hasting algorithm described in the lecture. Use the binary classification data binaryclass2.mat and the function laplacecomp. If you pass this function a 2-dimensional $\mathbf{w}$ vector, it will return $g(\mathbf{w};\mathbf{X},\mathbf{t},σ^2)$ and $logg(\mathbf{w};\mathbf{X},\mathbf{t},σ^2)$. 

(Remember that $g(\mathbf{w}; \mathbf{X}, \mathbf{t}, σ^2) \propto p(\mathbf{w}|\mathbf{X}, \mathbf{t}, σ^2)$, the posterior density of interest.)

## Posterior approximation

Let's write and test the function to compute $g(\mathbf{w}; \mathbf{X}, \mathbf{t}, σ^2)

In [2]:
def laplacecomp(w, X, t):
    '''
        Return two np array for g and log(g)
    '''
    w = np.array(w)
    X = np.array(X)
    t = np.array(t)

    # Computes g and log g for the laplace model introduced in the lecture.
    ss = 10 # Prior variance
    # Evaluate log prior
    logg = -(1/(2 * ss)) * w.transpose().dot(w)
    # Compute P
    P = np.array(1. / ( 1 + np.exp(-X.dot(w))))
    #print(P)
    logl = sum((t * np.log(P)) + ((1 - t) * np.log(1-P)))
    
    logg = logg + logl
    g = np.exp(logg)
    return (g[0][0], logg[0][0])

w = np.array([[-1],[-2]])
X = np.matrix([[0, 0], [1,1]])
t = np.array([0,1])
assert laplacecomp(w, X, t) == (0.37093273795143539, -0.99173453213368734), 'Error in laplacomp function'

## Problem visualisation
Let's start to work on our test case by importing the data and visualising the structure of this classification problem.

In [27]:
mat = sio.loadmat('binaryclass2.mat')
X = mat['X']
t = mat['t']
w = np.zeros([X.shape[1], 1])

df = pd.DataFrame(X)
df0 = pd.DataFrame(t)
df.insert(2, 2, value=df0)

trace1 = go.Scatter(x = df[df[2] == 0][0], y = df[df[2] == 0][1], mode = 'markers', name = 'Class 0')
trace2 = go.Scatter(x = df[df[2] == 1][0], y = df[df[2] == 1][1], mode = 'markers', name = 'Class 1')
data = [trace1, trace2]
py.iplot(data, filename='basic-scatter')

Let's write our Metropolis-Hastings for MCMC sampling.

In [32]:
def MH(X, t, total_trials, step):
    s = 0
    w = np.random.randn(2,1)
    ws = [w, ] 

    for s in np.arange(total_trials):
        #s += 1
        if (s+1) % (total_trials/100) == 0:
            if (s+1) % (total_trials/10) == 0:
                print('|', end='')
            else:
                print('.', end='')
                
        wp = np.random.randn(2,1) * 0.5 + w 
        w = ws[-1]
        gws, loggws = laplacecomp(w, X, t)
        gwp, loggwp = laplacecomp(wp, X, t)
        #compute acceptance ratio r
        logr = loggwp - loggws
        r = np.exp(logr)
    
        if logr >= 0:
            ws.append(wp) #acceptance
        else:
            u = np.random.uniform(0,1)
            if u <= r:
                ws.append(wp) #acceptance
            else:
                ws.append(w) #rejection
    print()
    return ws

Let's run our simulation 10000 times.

In [45]:
step = np.array([[0.5, 0], [0, 0.5]])
num_trials = 20000
samples = MH(X, t, num_trials, step)
#pprint(samples[0:10])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning:

invalid value encountered in multiply



.........|.........|.........|.........|.........|.........|.........|.........|.........|.........|


In [49]:
w_df = pd.DataFrame([np.array(samples)[:,0][:,0], np.array(samples)[:,1][:,0]]).transpose()

#colorscale = [ 'rgb(165,0,38)', 'rgb(215,48,39)', 'rgb(244,109,67)', 'rgb(253,174,97)', 'rgb(254,224,144)',\
#                'rgb(224,243,248)', 'rgb(171,217,233)', 'rgb(116,173,209)', 'rgb(69,117,180)',\
#                'rgb(49,54,149)']

colorscale2 = [ 'rgb(165,0,38)',  'rgb(244,109,67)',  'rgb(254,224,144)',\
                 'rgb(171,217,233)',  'rgb(69,117,180)',\
                'rgb(49,54,149)']

fig = ff.create_2d_density(
    w_df[0], w_df[1], colorscale=colorscale2,
    hist_color='rgb(255, 237, 222)', point_size=1
)

py.iplot(fig, filename='histogram_subplots')


This is the result of sampling done before. As we can clearly see, the strenght of this method is that can work without any prior assumption on the type of distribution as it's a non-parametric model for distribution density estimation.

Now, the next thing to do is to predict the new label of a $x_{new}$ starting from this samples list.

In [22]:
def predict(samples, x_new):
    p = 0.
    for w in samples:
        p += 1 / (1 + np.exp(-w.T.dot(x_new)))
    p /= len(samples)
    return p[0]

In [50]:
x_new = np.array([2,-4])
predict(samples, x_new)

0.23122952659698856

To have a more deep visualisation of the predictions, we can plot the probability for a point to be assigned to class 1.

In [24]:
xmin, xmax, xstep = (-6, 6.5, 0.5)
ymin, ymax, ystep = (-6, 6.5, 0.5)

p = []
for y in np.arange(ymin, ymax, ystep):
    p_x = []
    for x in np.arange(xmin, xmax, xstep):
        x_new = np.array([x, y])
        p_x.append(predict(samples, x_new))
    p.append(p_x)

In [25]:
prob = [go.Contour(
            z=p,
            y=np.arange(xmin, xmax, xstep),
            x=np.arange(ymin, ymax, ystep),
            contours=dict(coloring='heatmap')
    ), go.Scatter(
        x = df[df[2] == 0][0],
        y = df[df[2] == 0][1],
        mode = 'markers',
        marker = dict(color='orange'),
        name = 'Class 0'
    ), go.Scatter(
        x = df[df[2] == 1][0],
        y = df[df[2] == 1][1],
        mode = 'markers',
        marker = dict(color='blue'),
        name = 'Class 1'
)]

py.iplot(prob, filename='contour-scatter')

This is very interesting for a couple of reason:
- we achived a non-linear classification without using non-linear decision rule
- our belif on the label follows with less uncertanty the space region where the training data lays.